In [147]:
import pandas as pd, xarray as xr
import os, glob, re

study_nm = "la-fires"
study_desc = "Los Angeles wildfires 2025"
extent = "coastal southern California"


desc1 = {"dc7-doy-wrapped": "day of year on which drought ended",
         "fwi1x" : "peak January FWI",
         "spi-ond": "October-December SPI",
         "nino34det" : "OND detrended Nino3.4 index"}

desc2 = {"dc7-doy-wrapped": "dc-doy [doy] DOY after September 1st on which single largest 1-week drop in drought code occurred in "+extent,
         "fwi1x" : "fwi1x [] January maximum of one-day FWI in "+extent,
         "spi-ond": "spi-ond [] SPI of October-December mean rainfall (calibrated against 1991-2020) in "+extent,
         "nino34det": "nino34-ond [degC] Detrended Nino34 index averaged over October-December"}

short_varnm = {"dc7-doy-wrapped": "dc-doy",
               "fwi1x": "fwi1x",
               "spi-ond": "spi-ond",
               "nino34det": "nino34-ond"}

# Observations

In [148]:
flist = ["ts/nino34_ond.csv", "ts/era5_fwi1x-jan_socal-baja.csv", "ts/era5_spi-ond_socal-baja.csv", "ts/era5_drought-end-date.csv"]

In [160]:
for fnm in flist:
    
    varnm = {"ts/nino34_ond.csv" : "nino34det", 
             "ts/era5_fwi1x-jan_socal-baja.csv" : "fwi1x", 
             "ts/era5_spi-ond_socal-baja.csv" : "spi-ond", 
             "ts/era5_drought-end-date.csv" : "dc7-doy-wrapped"}[fnm]
    
    if fnm == "ts/era5_drought-end-date.csv":
        df = pd.read_csv(fnm).rename(columns = {"season_start_year" : "year", "biggest_DC_drop_from_1st_Sept" : "doy"})
        df = df.set_index("year")[["doy"]]
    else:
        df = pd.read_csv(fnm, index_col = "year")

    csv_fnm = study_nm+"_"+short_varnm[varnm]+"_ERA5"
    
    csv_fullpath = "ts-final/"+csv_fnm+".dat"
    if len(csv_fnm) > 61: print("! Filename too long: ", csv_fnm)
        
    # create extra header lines for upload to Climate Explorer 
    str1 = "# contact :: ERA5 "+desc1[varnm]+" - "+study_desc+", c.barnes22@imperial.ac.uk"
    str2 = "# "+desc2[varnm]+" in "+extent+" at "+csv_fnm+".dat"
    head = "# year "+short_varnm[varnm]
    
    # make .dat file
    ! echo "$str1 " > $csv_fullpath
    ! echo "$str2" >> $csv_fullpath
    ! echo "$head" >> $csv_fullpath
    df.to_csv(csv_fullpath, sep = " ", mode = "a", header = False)

In [87]:
varnm = "dc7-doy-wrapped"

for fnm in glob.glob("ts/cordex_"+varnm+"_*.csv"):
    
    mdl = "_".join(fnm[:-4].split("_")[2:])
    df = pd.read_csv(fnm, index_col = "year")
    
    csv_fnm = study_nm+"_"+short_varnm[varnm]+"_"+mdl
    
    csv_fullpath = "ts-final/"+csv_fnm+".dat"
    if len(csv_fnm) > 61: print("! Filename too long: ", csv_fnm)
        
    # create extra header lines for upload to Climate Explorer 
    str1 = "# contact :: CORDEX "+mdl+" "+desc1[varnm]+" - "+study_desc+", c.barnes22@imperial.ac.uk"
    str2 = "# "+desc2[varnm]+" in "+extent+" at "+csv_fnm+".dat"
    head = "# year "+short_varnm[varnm]
    
    # make .dat file
    ! echo "$str1 " > $csv_fullpath
    ! echo "$str2" >> $csv_fullpath
    ! echo "$head" >> $csv_fullpath
    df.to_csv(csv_fullpath, sep = " ", mode = "a", header = False)

# CMIP5 (CORDEX global covariates)


In [104]:
varnm = "nino34det"

for fnm in glob.glob("ts/nino34det_cmip5_*.nc"):
    
    mdl = "_".join(fnm[:-3].split("_")[-1:])
    df = pd.read_csv(fnm, sep = " ", index_col = "year")
    
    csv_fnm = study_nm+"_nino34-det_cmip5_"+mdl
    
    csv_fullpath = "ts-final/"+csv_fnm+".dat"
    if len(csv_fnm) > 61: print("! Filename too long: ", csv_fnm)
        
    # create extra header lines for upload to Climate Explorer 
    str1 = "# contact :: CMIP5 "+mdl+" OND detrended Nino3.4 index - "+study_desc+", c.barnes22@imperial.ac.uk"
    str2 = "# nino34-ond [degC] Detrended Nino34 index averaged over October-December at "+csv_fnm+".dat"
    head = "# year nino34-ond"
    
    # make .dat file
    ! echo "$str1 " > $csv_fullpath
    ! echo "$str2" >> $csv_fullpath
    ! echo "$head" >> $csv_fullpath
    df.to_csv(csv_fullpath, sep = " ", mode = "a", header = False)

# HighResMIP

In [134]:
for varnm in ["spi-ond", "dc7-doy-wrapped"]:

    for fnm in glob.glob("ts/spi-ond_highresSST*.csv"):

        mdl = "_".join(fnm[:-4].split("_")[2:])
        df = pd.read_csv(fnm, index_col = "year")

        csv_fnm = study_nm+"_"+short_varnm[varnm]+"_highresSST_"+mdl

        csv_fullpath = "ts-final/"+csv_fnm+".dat"
        if len(csv_fnm) > 61: print("! Filename too long: ", csv_fnm)

        # create extra header lines for upload to Climate Explorer 
        str1 = "# contact :: HighResSST "+mdl+" "+desc1[varnm]+" - "+study_desc+", c.barnes22@imperial.ac.uk"
        str2 = "# "+desc2[varnm]+" in "+extent+" at "+csv_fnm+".dat"
        head = "# year "+short_varnm[varnm]

        # make .dat file
        ! echo "$str1 " > $csv_fullpath
        ! echo "$str2" >> $csv_fullpath
        ! echo "$head" >> $csv_fullpath
        df.to_csv(csv_fullpath, sep = " ", mode = "a", header = False)

In [131]:
for fnm in glob.glob("ts/spi-ond_IPSL*.csv"):

    mdl = fnm.split("_")[1]+"_"+fnm[-12:-4]
    df = pd.read_csv(fnm, index_col = "year")
    
    csv_fnm = study_nm+"_"+short_varnm[varnm]+"_highresSST_"+mdl
    
    csv_fullpath = "ts-final/"+csv_fnm+".dat"
    if len(csv_fnm) > 61: print("! Filename too long: ", csv_fnm)
        
    # create extra header lines for upload to Climate Explorer 
    str1 = "# contact :: HighResSST "+mdl+" "+desc1[varnm]+" - "+study_desc+", c.barnes22@imperial.ac.uk"
    str2 = "# "+desc2[varnm]+" in "+extent+" at "+csv_fnm+".dat"
    head = "# year "+short_varnm[varnm]
    
    # make .dat file
    ! echo "$str1 " > $csv_fullpath
    ! echo "$str2" >> $csv_fullpath
    ! echo "$head" >> $csv_fullpath
    df.to_csv(csv_fullpath, sep = " ", mode = "a", header = False)